In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# cpg.dot들이 저장되어 있는 폴더 경로
folder_path = "/content/drive/MyDrive/test/"

# 폴더 내의 파일 목록 얻기
file_list = os.listdir(folder_path)

# DOT 파일을 읽는 함수 정의
def read_dot_file(file_path):
    with open(file_path, "r") as dot_file:
        lines = dot_file.readlines()
    return lines

In [ ]:
!pip install node2vec
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("codeparrot/codecomplex")

In [ ]:
def label_complexity(complexity):
    if complexity == 'constant':
        return 0
    elif complexity == 'logn':
        return 1
    elif complexity == 'linear':
        return 2
    elif complexity == 'nlogn':
        return 3
    else:
        return -1  # 알 수 없는 경우


In [ ]:
import numpy as np
from gensim.models import Word2Vec
import networkx as nx
from random import shuffle
from node2vec import Node2Vec  # Node2Vec 모듈 가져오기

#노드 정보 추출 및 벡터화 생성 코드

In [ ]:
def extract_node_info(lines):
    is_node = True
    nodes = {}
    for line in lines:
        line = line.strip()
        if line.startswith('digraph'):
            continue  # 그래프 시작 라인 무시
        if '->' in line:
            is_node = False
            continue
        if line == '}':
            break  # 그래프 정의 끝나면 종료
        if is_node:
            parts = line.split('[label = ')
            node_id = parts[0].strip('" ')
            label = parts[1].strip('" ]')
            nodes[node_id] = label
        elif line.endswith('{'):
            is_node = True

    return nodes


In [ ]:
import networkx as nx
import numpy as np
from node2vec import Node2Vec

# 함수로 노드 임베딩 생성
def generate_node_embeddings(graph_edges, node_labels, dimensions=64, walk_length=30, num_walks=200, workers=4):
    # 그래프 생성 및 노드 추가
    G = nx.Graph()

    for node_id, label in node_labels.items():
        G.add_node(node_id, label=label)

    # Node2Vec를 사용하여 노드 임베딩 생성
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # 임베딩 데이터 추출
    node_vectors = np.array([model.wv[node_id] for node_id in node_labels.keys()])

    return node_vectors



In [ ]:
# 노드 정보 추출 함수 호출
nodes = extract_node_info(lines)

# "IDENTIFIER" 또는 "METHOD" 레이블을 가진 노드 추출
matching_nodes = {node_id: label for node_id, label in nodes.items() if "CONTROL_STRUCTURE" in label or "IDENTIFIER" in label or "LITERAL" in label or "LOCAL" in label or "operator" in label}

# 노드 임베딩 생성 함수 호출
node_vectors = generate_node_embeddings(edges, matching_nodes)

print(node_vectors)

#엣지 정보 추출 및 벡터화 코드

In [ ]:
def extract_edges_from_dot_file(lines):
    edges = []

    for line in lines:
        line = line.strip()
        if line.startswith('digraph'):
            continue  # 그래프 시작 라인 무시
        if '->' in line:
            parts = line.split('[ label = "')
            if len(parts) > 1:
                edge_labels = parts[1].split('"]')
                edge_type = edge_labels[0].strip()
                if edge_type.startswith("CFG:") or edge_type.startswith("DDG:") or edge_type.startswith("CDG:"):
                    # 엣지 정보 추출
                    source, target = map(str.strip, parts[0].split('->'))
                    if edge_type.startswith("CFG:"):
                        edges.append((source, target))
                    elif edge_type.startswith("DDG:"):
                        edges.append((source, target))
                    else:
                        edges.append((source, target))

    return edges
# 함수 호출하여 엣지 정보 추출


In [ ]:
def generate_edge_vectors(edges, vector_size=64, window=10, min_count=1, sg=1, workers=4, epochs=10):
    # 엣지를 문자열 리스트로 변환
    edge_strings = [f"{edge[0]} -> {edge[1]}" for edge in edges]

    # Word2Vec 모델 생성
    model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=workers)

    # 모델에 학습 데이터 추가
    model.build_vocab(corpus_iterable=[edge_strings])

    # 모델 학습
    model.train(corpus_iterable=[edge_strings], total_examples=len(edge_strings), epochs=epochs)

    # 엣지 벡터를 NumPy 배열로 변환
    edge_vectors = np.array([model.wv[edge_str] for edge_str in edge_strings])

    return edge_vectors


In [ ]:

edges = extract_edges_from_dot_file(lines)
edge_vectors = generate_edge_vectors(edges)
print(edge_vectors)

#데이터셋 구축


In [ ]:
import os
import re
import csv

# 폴더 내의 각 파일을 읽어서 처리
idx = 0

# CSV 파일 열기
with open('/content/drive/MyDrive/code_data.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Index', 'Node_Vectors', 'Edge_Vectors', 'Code_Complexity'])
    for file_name in file_list:
        if file_name.endswith(".dot"):  # 확장자가 .dot인 파일만 처리
            file_path = os.path.join(folder_path, file_name)
            lines = read_dot_file(file_path)
            print(file_name)
            # 노드 정보 추출 함수 호출
            nodes = extract_node_info(lines)
            # "IDENTIFIER" 또는 "METHOD" 레이블을 가진 노드 추출
            matching_nodes = {node_id: label for node_id, label in nodes.items() if "CONTROL_STRUCTURE" in label or "IDENTIFIER" in label or "LITERAL" in label or "LOCAL" in label or "operator" in label}
            # 노드 임베딩 생성 함수 호출
            node_vectors = generate_node_embeddings(edges, matching_nodes)

            # 엣지 정보 추출 및 임베딩 생성 함수 호출
            edges = extract_edges_from_dot_file(lines)
            edge_vectors = generate_edge_vectors(edges)
            vec_edges = []
            for edge in edge_vectors:
                vec_edges.append(edge.tolist())

            #시간복잡도 값 추출
            match = re.match(r'^(\d+)', file_name)
            if match:
                number_value = int(match.group(1))
                codecomplexity = label_complexity(dataset['train'][number_value]['complexity'])
            else:
                codecomplexity = -1

            writer.writerow([idx, node_vectors, vec_edges, codecomplexity])
            idx += 1
            # lines를 이용한 추가 작업 수행



In [ ]:
for file_name in file_list:
    if file_name.endswith(".dot"):  # 확장자가 .dot인 파일만 처리
        file_path = os.path.join(folder_path, file_name)
        lines = read_dot_file(file_path)
        print(file_name)
        # 노드 정보 추출 함수 호출
        nodes = extract_node_info(lines)
        # "IDENTIFIER" 또는 "METHOD" 레이블을 가진 노드 추출
        matching_nodes = {node_id: label for node_id, label in nodes.items() if "CONTROL_STRUCTURE" in label or "IDENTIFIER" in label or "LITERAL" in label or "LOCAL" in label or "operator" in label}
        # 노드 임베딩 생성 함수 호출
        node_vectors = generate_node_embeddings(edges, matching_nodes)

        # 엣지 정보 추출 및 임베딩 생성 함수 호출
        edges = extract_edges_from_dot_file(lines)
        edge_vectors = generate_edge_vectors(edges)
        vec_edges = []
        for edge in edge_vectors:
             vec_edges.append(edge.tolist())

        #시간복잡도 값 추출
        match = re.match(r'^(\d+)', file_name)
        if match:
            number_value = int(match.group(1))
            codecomplexity = label_complexity(dataset['train'][number_value]['complexity'])
        else:
            codecomplexity = -1

        writer.writerow([idx, node_vectors, vec_edges, codecomplexity])
        idx += 1
        # lines를 이용한 추가 작업 수행
